In [ ]:
import numpy as np  # Importing the NumPy library for numerical operations
import os  # Importing the OS library for interacting with the operating system
import imageio  # Importing the imageio library for reading and writing image data
from skimage.transform import resize  # Importing the resize function from skimage.transform for resizing images
import datetime  # Importing the datetime module to work with date and time
import warnings  # Importing the warnings module to manage warnings
warnings.filterwarnings("ignore")  # Ignoring all warnings
import abc  # Importing the abc module for abstract base classes
from sys import getsizeof  # Importing getsizeof from sys module to get the size of an object

In [ ]:
np.random.seed(30)  # Setting the seed for NumPy's random number generator for reproducibility
import random as rn  # Importing the random library and aliasing it as rn
rn.seed(30)  # Setting the seed for Python's random number generator for reproducibility
from keras import backend as K  # Importing the backend module from Keras and aliasing it as K
import tensorflow as tf  # Importing TensorFlow and aliasing it as tf
tf.random.set_seed(30)  # Setting the seed for TensorFlow's random number generator for reproducibility

In [ ]:
import cv2  # Importing the OpenCV library for computer vision tasks
import matplotlib.pyplot as plt  # Importing Matplotlib's pyplot module for plotting and aliasing it as plt
#%matplotlib inline  # Ensuring that Matplotlib plots are displayed inline within Jupyter notebooks

In [ ]:
from keras.models import Sequential, Model  # Importing the Sequential and Model classes from Keras for building models
from keras.layers import Dense, GRU, Flatten, TimeDistributed, BatchNormalization, Activation  # Importing various layers from Keras
from keras.layers import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D  # Importing convolutional and pooling layers for 2D and 3D data
from keras.layers import LSTM  # Importing the LSTM layer for recurrent neural networks
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping  # Importing callback functions for training
# from keras import optimizers  # Commented out import statement for Keras optimizers
from tensorflow.keras import optimizers  # Importing optimizers from TensorFlow's Keras
from keras.layers import Dropout  # Importing the Dropout layer for regularizatio

In [ ]:
def plot(history):
    # Function to plot the training and validation loss and accuracy

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))  # Creating a figure with 2 subplots side by side
    axes[0].plot(history.history['loss'])  # Plotting training loss on the first subplot
    axes[0].plot(history.history['val_loss'])  # Plotting validation loss on the first subplot
    axes[0].grid()  # Adding grid to the first subplot
    axes[0].legend(['loss', 'val_loss'])  # Adding legend to the first subplot

    axes[1].plot(history.history['categorical_accuracy'])  # Plotting training accuracy on the second subplot
    axes[1].plot(history.history['val_categorical_accuracy'])  # Plotting validation accuracy on the second subplot
    axes[1].grid()  # Adding grid to the second subplot
    axes[1].legend(['categorical_accuracy', 'val_categorical_accuracy'])  # Adding legend to the second subplot

In [ ]:
class ModelBuilder(metaclass=abc.ABCMeta):
    # Method to initialize paths where project data resides
    def initialize_path(self, project_folder):
        self.train_doc = np.random.permutation(open(project_folder + '/' + 'train.csv').readlines())  # Shuffle training data
        self.val_doc = np.random.permutation(open(project_folder + '/' + 'val.csv').readlines())  # Shuffle validation data
        self.train_path = project_folder + '/' + 'train'  # Path to training data
        self.val_path = project_folder + '/' + 'val'  # Path to validation data
        self.num_train_sequences = len(self.train_doc)  # Number of training sequences
        self.num_val_sequences = len(self.val_doc)  # Number of validation sequences

    # Method to initialize image properties
    def initialize_image_properties(self, image_height=100, image_width=100):
        self.image_height = image_height  # Height of the image
        self.image_width = image_width  # Width of the image
        self.channels = 3  # Number of image channels (RGB)
        self.num_classes = 5  # Number of classes for classification
        self.total_frames = 30  # Total number of frames in each video

    # Method to initialize hyperparameters: batch size, frames to sample, and number of epochs
    def initialize_hyperparams(self, frames_to_sample=30, batch_size=20, num_epochs=20):
        self.frames_to_sample = frames_to_sample  # Number of frames to sample from each video
        self.batch_size = batch_size  # Batch size for training
        self.num_epochs = num_epochs  # Number of epochs for training

    # Generator function to yield batches of data
    def generator(self, source_path, folder_list, augment=False):
        img_idx = np.round(np.linspace(0, self.total_frames - 1, self.frames_to_sample)).astype(int)  # Indices of frames to sample
        batch_size = self.batch_size  # Batch size for training
        while True:
            t = np.random.permutation(folder_list)  # Shuffle folder list
            num_batches = len(t) // batch_size  # Number of batches per epoch

            for batch in range(num_batches):
                batch_data, batch_labels = self.one_batch_data(source_path, t, batch, batch_size, img_idx, augment)  # Get one batch of data
                yield batch_data, batch_labels  # Yield batch data and labels

            remaining_seq = len(t) % batch_size  # Remaining sequences if not divisible by batch size

            if remaining_seq != 0:
                batch_data, batch_labels = self.one_batch_data(source_path, t, num_batches, batch_size, img_idx, augment, remaining_seq)  # Get remaining sequences
                yield batch_data, batch_labels  # Yield remaining batch data and labels

    # Method to get data for one batch
    def one_batch_data(self, source_path, t, batch, batch_size, img_idx, augment, remaining_seq=0):
        seq_len = remaining_seq if remaining_seq else batch_size  # Length of the sequence (batch size or remaining sequences)

        batch_data = np.zeros((seq_len, len(img_idx), self.image_height, self.image_width, self.channels))  # Initialize batch data array
        batch_labels = np.zeros((seq_len, self.num_classes))  # Initialize batch labels array

        if augment:
            batch_data_aug = np.zeros((seq_len, len(img_idx), self.image_height, self.image_width, self.channels))  # Initialize augmented data array

        for folder in range(seq_len):
            imgs = os.listdir(source_path + '/' + t[folder + (batch * batch_size)].split(';')[0])  # List of images in the folder
            for idx, item in enumerate(img_idx):
                # Reading and resizing the image
                image = imageio.imread(source_path + '/' + t[folder + (batch * batch_size)].strip().split(';')[0] + '/' + imgs[item]).astype(np.float32)
                image_resized = resize(image, (self.image_height, self.image_width, 3))

                # Normalizing the images
                batch_data[folder, idx, :, :, 0] = (image_resized[:, :, 0]) / 255
                batch_data[folder, idx, :, :, 1] = (image_resized[:, :, 1]) / 255
                batch_data[folder, idx, :, :, 2] = (image_resized[:, :, 2]) / 255

                if augment:
                    # Applying random shifts to the image
                    shifted = cv2.warpAffine(image,
                                             np.float32([[1, 0, np.random.randint(-30, 30)], [0, 1, np.random.randint(-30, 30)]]),
                                             (image.shape[1], image.shape[0]))

                    gray = cv2.cvtColor(shifted, cv2.COLOR_BGR2GRAY)  # Convert to grayscale

                    x0, y0 = np.argwhere(gray > 0).min(axis=0)
                    x1, y1 = np.argwhere(gray > 0).max(axis=0)
                    # Cropping the images to target gestures and remove noise
                    cropped = shifted[x0:x1, y0:y1, :]

                    image_resized = resize(cropped, (self.image_height, self.image_width, 3))

                    # Normalizing the augmented images
                    batch_data_aug[folder, idx, :, :, 0] = (image_resized[:, :, 0]) / 255
                    batch_data_aug[folder, idx, :, :, 1] = (image_resized[:, :, 1]) / 255
                    batch_data_aug[folder, idx, :, :, 2] = (image_resized[:, :, 2]) / 255

            batch_labels[folder, int(t[folder + (batch * batch_size)].strip().split(';')[2])] = 1  # One-hot encoding the labels

        if augment:
            batch_data = np.concatenate([batch_data, batch_data_aug])  # Concatenating original and augmented data
            batch_labels = np.concatenate([batch_labels, batch_labels])  # Concatenating original and augmented labels

        return batch_data, batch_labels  # Returning batch data and labels

    # Method to train the model
    def train_model(self, model, augment_data=False):
        train_generator = self.generator(self.train_path, self.train_doc, augment=augment_data)  # Training data generator
        val_generator = self.generator(self.val_path, self.val_doc)  # Validation data generator

        model_name = 'model_init' + '_' + str(datetime.datetime.now()).replace(' ', '').replace(':', '_') + '/'  # Model name with timestamp

        if not os.path.exists(model_name):
            os.mkdir(model_name)  # Create directory for saving the model

        filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.keras'  # Filepath for saving the model
        # As per the req saving files in .h5 format
      #  filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'  # Filepath for saving the model

        # Callbacks for saving the best model and reducing learning rate on plateau
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
        LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)

        # earlystop = EarlyStopping( monitor="val_loss", min_delta=0, patience=10, verbose=1)
        # callbacks_list = [checkpoint, LR, earlystop]
        callbacks_list = [checkpoint, LR]  # List of callbacks

        if (self.num_train_sequences % self.batch_size) == 0:
            steps_per_epoch = int(self.num_train_sequences / self.batch_size)  # Calculate steps per epoch
        else:
            steps_per_epoch = (self.num_train_sequences // self.batch_size) + 1

        if (self.num_val_sequences % self.batch_size) == 0:
            validation_steps = int(self.num_val_sequences / self.batch_size)  # Calculate validation steps
        else:
            validation_steps = (self.num_val_sequences // self.batch_size) + 1

        history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=self.num_epochs, verbose=1,
                            callbacks=callbacks_list, validation_data=val_generator,
                            validation_steps=validation_steps, class_weight=None, initial_epoch=0)  # Train the model
        return history  # Return training history

    @abc.abstractmethod
    def define_model(self):
        pass  # Abstract method to define the model architecture

# ModelBuilder Class Explanation

The **`ModelBuilder`** class is a base class for building deep learning models, especially for tasks like gesture recognition and video classification. It provides methods to initialize paths, image properties, hyperparameters, data generators, and model training procedures. Below is a detailed explanation of each part of the code:

---

### **Methods and Their Purposes**

1. **`initialize_path`**:
   - Sets up paths for training and validation data.
   - Shuffles the training and validation data (`train.csv` and `val.csv`).
   - Initializes the number of training and validation sequences.

2. **`initialize_image_properties`**:
   - Sets image height, width, and number of channels (default: RGB).
   - Specifies the number of classes (`num_classes`) and total frames in each video (`total_frames`).

3. **`initialize_hyperparams`**:
   - Configures key training parameters:
     - `frames_to_sample`: Number of frames to sample from each video.
     - `batch_size`: Size of each training/validation batch.
     - `num_epochs`: Total epochs for training.

4. **`generator`**:
   - Implements a generator to yield batches of data for training and validation.
   - Handles data shuffling and ensures efficient memory usage by generating batches on the fly.
   - Supports data augmentation when `augment=True`.

5. **`one_batch_data`**:
   - Extracts data for a single batch:
     - Reads images from the specified paths.
     - Resizes and normalizes images for model input.
     - Supports data augmentation with random shifts, cropping, and grayscale conversion.
   - Implements one-hot encoding for labels.
   - Handles cases where the number of sequences is not divisible by the batch size.

6. **`train_model`**:
   - Trains the model using the provided generator:
     - Sets up callbacks, including:
       - **`ModelCheckpoint`**: Saves the best model during training.
       - **`ReduceLROnPlateau`**: Reduces the learning rate when the validation loss plateaus.
     - Dynamically calculates `steps_per_epoch` and `validation_steps` based on the number of sequences and batch size.
   - Returns the training history for performance analysis.

7. **`define_model`**:
   - Abstract method to be implemented by subclasses to define the specific model architecture.

---

### **Notable Features**

1. **Dynamic Generators**:
   - Efficiently loads data in batches, avoiding memory issues for large datasets.
   - Augmentation enhances model generalization.

2. **Callbacks**:
   - **`ModelCheckpoint`**: Saves models during training to avoid loss of progress.
   - **`ReduceLROnPlateau`**: Adjusts learning rate dynamically to improve convergence.

3. **Data Preprocessing**:
   - Normalizes pixel values to [0, 1].
   - Supports augmentation techniques like random shifts and cropping.

4. **Flexible Architecture**:
   - Abstract `define_model` method allows subclasses to define various neural network architectures.

---

### **Use Case**
This class is highly modular and reusable for video-related machine learning tasks. By subclassing and implementing the `define_model` method, developers can quickly build and train custom models with different architectures like CNNs, RNNs, or transfer learning-based models.


In [ ]:
class ModelConv3D1(ModelBuilder):

    def define_model(self):
        # Define the 3D convolutional neural network model

        model = Sequential()  # Initialize the model as a sequential model

        # First 3D convolutional layer
        model.add(Conv3D(16, (3, 3, 3), padding='same', input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        model.add(Activation('relu'))  # ReLU activation
        model.add(BatchNormalization())  # Batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling

        # Second 3D convolutional layer
        model.add(Conv3D(32, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))  # ReLU activation
        model.add(BatchNormalization())  # Batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling

        # Third 3D convolutional layer
        model.add(Conv3D(64, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))  # ReLU activation
        model.add(BatchNormalization())  # Batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling

        # Fourth 3D convolutional layer
        model.add(Conv3D(128, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))  # ReLU activation
        model.add(BatchNormalization())  # Batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling

        model.add(Flatten())  # Flatten the output from the convolutional layers

        # First dense (fully connected) layer
        model.add(Dense(128, activation='relu'))  # Dense layer with ReLU activation
        model.add(BatchNormalization())  # Batch normalization
        model.add(Dropout(0.5))  # Dropout for regularization

        # Second dense (fully connected) layer
        model.add(Dense(64, activation='relu'))  # Dense layer with ReLU activation
        model.add(BatchNormalization())  # Batch normalization
        model.add(Dropout(0.25))  # Dropout for regularization

        # Output layer
        model.add(Dense(self.num_classes, activation='softmax'))  # Output layer with softmax activation

        optimiser = optimizers.Adam()  # Using Adam optimizer
        # optimiser = 'sgd'  # Alternative optimizer (commented out)

        # Compile the model
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model  # Return the compiled model

# ModelConv3D1 Class Explanation

The **`ModelConv3D1`** class is a subclass of the `ModelBuilder` base class. It defines a 3D Convolutional Neural Network (CNN) architecture for tasks involving spatiotemporal data, such as video classification. Below is a detailed explanation of the code:

---

### **Model Architecture**

1. **Initialization**:
   - The model is initialized as a **`Sequential`** model, allowing layers to be stacked sequentially.

2. **Convolutional Layers**:
   - The model includes four 3D convolutional layers (`Conv3D`) to capture spatial and temporal features:
     - **Input Shape**: `(frames_to_sample, image_height, image_width, channels)`.
     - **Filters**:
       - 16 filters in the first layer.
       - 32 filters in the second layer.
       - 64 filters in the third layer.
       - 128 filters in the fourth layer.
     - **Kernel Size**: `(3, 3, 3)` for the first layer, and `(2, 2, 2)` for subsequent layers.
     - **Padding**: `'same'` ensures the output dimensions remain the same after convolution.

3. **Activation and Normalization**:
   - **Activation**: Each convolutional layer uses the ReLU activation function to introduce non-linearity.
   - **BatchNormalization**: Normalizes the activations of each layer, speeding up convergence and improving stability.

4. **Pooling Layers**:
   - Each convolutional block is followed by a **MaxPooling3D** layer with a pool size of `(2, 2, 2)`, reducing the spatial and temporal dimensions to extract prominent features.

5. **Flatten Layer**:
   - The 3D outputs from the convolutional layers are flattened into a 1D vector for input into the fully connected layers.

6. **Dense Layers**:
   - The dense layers act as fully connected layers to learn high-level representations:
     - **First Dense Layer**:
       - 128 neurons.
       - ReLU activation, batch normalization, and 50% dropout.
     - **Second Dense Layer**:
       - 64 neurons.
       - ReLU activation, batch normalization, and 25% dropout.

7. **Output Layer**:
   - A final dense layer with `num_classes` neurons and **softmax activation** for multi-class classification.

---

### **Optimizer and Compilation**

- **Optimizer**: Adam optimizer is used for its adaptive learning rate and faster convergence.
- **Loss Function**: Categorical crossentropy is chosen for multi-class classification tasks.
- **Metrics**: The model tracks `categorical_accuracy` to evaluate performance during training and validation.

---

### **Key Features**

1. **3D Convolutions**:
   - Captures both spatial and temporal patterns in videos.

2. **Regularization**:
   - Dropout layers prevent overfitting.
   - Batch normalization improves training stability.

3. **Flexible Architecture**:
   - Designed to handle input videos with multiple frames and RGB channels.

4. **Modularity**:
   - The method can be easily reused and adapted by overriding the `define_model` method.

---

### **Use Case**
This model is ideal for video classification tasks where both spatial and temporal information are crucial, such as gesture recognition, action detection, and video tagging.


In [ ]:
conv_3d1 = ModelConv3D1()  # Instantiate the ModelConv3D1 class

conv_3d1.initialize_path(project_folder)  # Initialize paths where project data resides
conv_3d1.initialize_image_properties(image_height=160, image_width=160)  # Initialize image properties with specified height and width
conv_3d1.initialize_hyperparams(frames_to_sample=30, batch_size=10, num_epochs=1)  # Initialize hyperparameters with specified values

conv_3d1_model = conv_3d1.define_model()  # Define the model using the method from ModelConv3D1 class
conv_3d1_model.summary()  # Print the summary of the model architecture

# Explanation of the Code: Initializing and Summarizing ModelConv3D1

This code demonstrates the process of initializing a 3D CNN model (`ModelConv3D1`) and verifying its architecture by printing a summary. Below is a step-by-step explanation of the code:

---

### **Step-by-Step Explanation**

1. **Instantiate the `ModelConv3D1` Class**:
   ```python
   conv_3d1 = ModelConv3D1()
- Creates an object of the ModelConv3D1 class.
This object provides methods for setting up data paths, initializing image
- properties, defining hyperparameters, and building a 3D convolutional neural network model.
- 2.**Initialize Data Paths:**
conv_3d1.initialize_path(project_folder)
- Configures paths to the dataset.
- **Parameter:**
- 'project_folder:' Directory containing the project data (e.g., training and validation datasets in CSV and folder formats).
3.**Set Image Properties:**
conv_3d1.initialize_image_properties(image_height=160, image_width=160)
- Specifies the height and width of input images.
- **Parameters:**
- 'image_height:' The height of the resized input images (160 pixels).
- 'image_width:' The width of the resized input images (160 pixels).
- Ensures all images have consistent dimensions before being fed into the model.
4.**Initialize Hyperparameters:**
- conv_3d1.initialize_hyperparams(frames_to_sample=30, batch_size=10, num_epochs=1)
  
**Parameters:**
- frames_to_sample: Number of frames to sample from each video (30 frames).
- batch_size: Number of samples processed in each batch during training (10 samples).
- num_epochs: Number of times the model will iterate over the entire dataset (1 epoch).

5.**Define the Model:**
conv_3d1_model = conv_3d1.define_model()
- Calls the define_model method to build the 3D CNN architecture.
Includes layers like:
- Conv3D for extracting spatiotemporal features.
- MaxPooling3D for down-sampling the spatial dimensions.
- Fully connected layers for classification.























In [ ]:
test_generator = ModelConv3D1()  # Instantiate the ModelConv3D1 class for testing the generator

test_generator.initialize_path(project_folder)  # Initialize paths where project data resides
test_generator.initialize_image_properties(image_height=160, image_width=160)  # Initialize image properties with specified height and width
test_generator.initialize_hyperparams(frames_to_sample=30, batch_size=3, num_epochs=1)  # Initialize hyperparameters with specified values

g = test_generator.generator(test_generator.val_path, test_generator.val_doc, augment=True)  # Create a generator for validation data with augmentation
batch_data, batch_labels = next(g)  # Generate one batch of data and labels

fig, axes = plt.subplots(nrows=1, ncols=2)  # Create a figure with 2 subplots side by side
axes[0].imshow(batch_data[0, 15, :, :, :])  # Display the 16th frame of the first sample in the batch
axes[1].imshow(batch_data[3, 15, :, :, :])  # Display the 16th frame of the fourth sample in the batch
plt.show()  # Show the plots

In [ ]:
conv_3d1.train_model(conv_3d1_model)  # Train the model using the train_model method from the ModelConv3D1 class


In [ ]:
conv_3d1 = ModelConv3D1()  # Instantiate the ModelConv3D1 class

conv_3d1.initialize_path(project_folder)  # Initialize paths where project data resides
conv_3d1.initialize_image_properties(image_height=100, image_width=100)  # Initialize image properties with specified height and width
conv_3d1.initialize_hyperparams(frames_to_sample=30, batch_size=20, num_epochs=2)  # Initialize hyperparameters with specified values

conv_3d1_model = conv_3d1.define_model()  # Define the model using the method from ModelConv3D1 class

print("Total Params:", conv_3d1_model.count_params())  # Print the total number of parameters in the model

conv_3d1.train_model(conv_3d1_model)  # Train the model using the train_model method from the ModelConv3D1 class


In [ ]:
conv_3d1 = ModelConv3D1()  # Instantiate the ModelConv3D1 class

conv_3d1.initialize_path(project_folder)  # Initialize paths where project data resides
conv_3d1.initialize_image_properties(image_height=100, image_width=100)  # Initialize image properties with specified height and width
conv_3d1.initialize_hyperparams(frames_to_sample=30, batch_size=20, num_epochs=2)  # Initialize hyperparameters with specified values

conv_3d1_model = conv_3d1.define_model()  # Define the model using the method from ModelConv3D1 class

print("Total Params:", conv_3d1_model.count_params())  # Print the total number of parameters in the model

conv_3d1.train_model(conv_3d1_model)  # Train the model using the train_model method from the ModelConv3D1 class

In [ ]:
class ModelConv3D1(ModelBuilder):

    def define_model(self, filtersize=(3, 3, 3), dense_neurons=64, dropout=0.25):
        # Define the 3D convolutional neural network model with customizable parameters

        model = Sequential()  # Initialize the model as a sequential model

        # First 3D convolutional layer
        model.add(Conv3D(16, filtersize, padding='same', input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        model.add(Activation('relu'))  # ReLU activation
        model.add(BatchNormalization())  # Batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling

        # Second 3D convolutional layer
        model.add(Conv3D(32, filtersize, padding='same'))
        model.add(Activation('relu'))  # ReLU activation
        model.add(BatchNormalization())  # Batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling

        # Third 3D convolutional layer
        model.add(Conv3D(64, filtersize, padding='same'))
        model.add(Activation('relu'))  # ReLU activation
        model.add(BatchNormalization())  # Batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling

        # Fourth 3D convolutional layer
        model.add(Conv3D(128, filtersize, padding='same'))
        model.add(Activation('relu'))  # ReLU activation
        model.add(BatchNormalization())  # Batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling

        model.add(Flatten())  # Flatten the output from the convolutional layers

        # First dense (fully connected) layer
        model.add(Dense(dense_neurons, activation='relu'))  # Dense layer with ReLU activation
        model.add(BatchNormalization())  # Batch normalization
        model.add(Dropout(dropout))  # Dropout for regularization

        # Second dense (fully connected) layer
        model.add(Dense(dense_neurons, activation='relu'))  # Dense layer with ReLU activation
        model.add(BatchNormalization())  # Batch normalization
        model.add(Dropout(dropout))  # Dropout for regularization

        # Output layer
        model.add(Dense(self.num_classes, activation='softmax'))  # Output layer with softmax activation

        optimiser = optimizers.Adam()  # Using Adam optimizer

        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])  # Compile the model

        return model  # Return the compiled model

In [ ]:
conv_3d1 = ModelConv3D1()  # Instantiate the ModelConv3D1 class

conv_3d1.initialize_path(project_folder)  # Initialize paths where project data resides
conv_3d1.initialize_image_properties(image_height=160, image_width=160)  # Initialize image properties with specified height and width
conv_3d1.initialize_hyperparams(frames_to_sample=20, batch_size=40, num_epochs=20)  # Initialize hyperparameters with specified values

conv_3d1_model = conv_3d1.define_model()  # Define the model using the method from ModelConv3D1 class
conv_3d1_model.summary()  # Print the summary of the model architecture

In [ ]:
# Print the total number of parameters in the model
print("Total Params:", conv_3d1_model.count_params())

# Train the model using the train_model method from the ModelConv3D1 class
history_model1 = conv_3d1.train_model(conv_3d1_model)

In [ ]:
plot(history_model1)


In [ ]:
# Instantiate the ModelConv3D1 class
conv_3d2 = ModelConv3D1()

# Initialize paths where project data resides
conv_3d2.initialize_path(project_folder)

# Initialize image properties with specified height and width
conv_3d2.initialize_image_properties(image_height=160, image_width=160)

# Initialize hyperparameters with specified values
conv_3d2.initialize_hyperparams(frames_to_sample=20, batch_size=20, num_epochs=25)

# Define the model with customized dense layer neurons and dropout rate
conv_3d2_model = conv_3d2.define_model(dense_neurons=256, dropout=0.5)

# Print the summary of the model architecture
conv_3d2_model.summary()


In [ ]:
print("Total Params:", conv_3d2_model.count_params())
history_model2=conv_3d2.train_model(conv_3d2_model,augment_data=True)

In [ ]:
plot(history_model2)




In [ ]:
# Define a new class ModelConv3D3 inheriting from ModelBuilder
class ModelConv3D3(ModelBuilder):

    # Define a method to define the model architecture
    def define_model(self, filtersize=(3, 3, 3), dense_neurons=64, dropout=0.25):

        model = Sequential()  # Initialize a sequential model

        # Add the first 3D convolutional layer with specified parameters
        model.add(Conv3D(16, filtersize, padding='same', input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling

        # Add the second 3D convolutional layer with specified parameters
        model.add(Conv3D(32, filtersize, padding='same'))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling

        # Add the third 3D convolutional layer with specified parameters
        model.add(Conv3D(64, filtersize, padding='same'))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling

        # Add the fourth 3D convolutional layer with specified parameters
        model.add(Conv3D(128, filtersize, padding='same'))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling

        model.add(Flatten())  # Flatten the output from the convolutional layers

        # Add the first dense (fully connected) layer with specified number of neurons and activation function
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())  # Batch normalization
        model.add(Dropout(dropout))  # Dropout for regularization

        # Add the second dense (fully connected) layer with specified number of neurons and activation function
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())  # Batch normalization
        model.add(Dropout(dropout))  # Dropout for regularization

        # Add the output layer with softmax activation function
        model.add(Dense(self.num_classes, activation='softmax'))

        # Compile the model with Adam optimizer and specified learning rate
        optimiser = optimizers.Adam(lr=0.0002)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model  # Return the compiled model

In [ ]:
# Instantiate the ModelConv3D3 class
conv_3d3 = ModelConv3D3()

# Initialize paths where project data resides
conv_3d3.initialize_path(project_folder)

# Initialize image properties with specified height and width
conv_3d3.initialize_image_properties(image_height=120, image_width=120)

# Initialize hyperparameters with specified values
conv_3d3.initialize_hyperparams(frames_to_sample=16, batch_size=30, num_epochs=25)

# Define the model with customized filter size, dense layer neurons, and dropout rate
conv_3d3_model = conv_3d3.define_model(filtersize=(2, 2, 2), dense_neurons=256, dropout=0.5)

# Print the summary of the model architecture
conv_3d3_model.summary()

In [ ]:
print("Total Params:", conv_3d3_model.count_params())
history_model3=conv_3d3.train_model(conv_3d3_model,augment_data=True)

In [ ]:
plot(history_model3)


In [ ]:
# Adding more layers - Batch Size = 20 and No. of Epochs = 25
# Define a new class ModelConv3D4 inheriting from ModelBuilder
class ModelConv3D4(ModelBuilder):

    # Define a method to define the model architecture
    def define_model(self, filtersize=(3, 3, 3), dense_neurons=64, dropout=0.25):

        model = Sequential()  # Initialize a sequential model

        # First convolutional block
        model.add(Conv3D(16, filtersize, padding='same', input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization

        # Second convolutional block
        model.add(Conv3D(16, filtersize, padding='same', input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization

        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling

        # Third convolutional block
        model.add(Conv3D(32, filtersize, padding='same'))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization

        # Fourth convolutional block
        model.add(Conv3D(32, filtersize, padding='same'))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization

        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling

        # Fifth convolutional block
        model.add(Conv3D(64, filtersize, padding='same'))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization

        # Sixth convolutional block
        model.add(Conv3D(64, filtersize, padding='same'))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization

        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling

        # Seventh convolutional block
        model.add(Conv3D(128, filtersize, padding='same'))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization

        # Eighth convolutional block
        model.add(Conv3D(128, filtersize, padding='same'))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization

        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling

        model.add(Flatten())  # Flatten the output from the convolutional layers

        # First dense (fully connected) layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())  # Batch normalization
        model.add(Dropout(dropout))  # Dropout for regularization

        # Second dense (fully connected) layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())  # Batch normalization
        model.add(Dropout(dropout))  # Dropout for regularization

        # Output layer with softmax activation function
        model.add(Dense(self.num_classes, activation='softmax'))

        # Compile the model with Adam optimizer and default learning rate
        optimiser = optimizers.Adam()
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model  # Return the compiled model

In [ ]:
conv_3d4=ModelConv3D4()
conv_3d4.initialize_path(project_folder)
conv_3d4.initialize_image_properties(image_height=120,image_width=120)
conv_3d4.initialize_hyperparams(frames_to_sample=16,batch_size=20,num_epochs=10)
conv_3d4_model=conv_3d4.define_model(filtersize=(3,3,3),dense_neurons=256,dropout=0.5)
conv_3d4_model.summary()

In [ ]:
print("Total Params:", conv_3d4_model.count_params())
history_model4=conv_3d4.train_model(conv_3d4_model,augment_data=True)

In [ ]:
plot(history_model4)


In [ ]:
# Adding dropout at convolution layers

# Define a new class ModelConv3D5 inheriting from ModelBuilder
class ModelConv3D5(ModelBuilder):
    
    # Define a method to define the model architecture
    def define_model(self, filtersize=(3, 3, 3), dense_neurons=64, dropout=0.25):

        model = Sequential()  # Initialize a sequential model

        # First convolutional block
        model.add(Conv3D(16, filtersize, padding='same', input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization

        # Second convolutional block
        model.add(Conv3D(16, filtersize, padding='same', input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization

        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling
        model.add(Dropout(dropout))  # Dropout for regularization

        # Third convolutional block
        model.add(Conv3D(32, filtersize, padding='same'))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization

        # Fourth convolutional block
        model.add(Conv3D(32, filtersize, padding='same'))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization

        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling
        model.add(Dropout(dropout))  # Dropout for regularization

        # Fifth convolutional block
        model.add(Conv3D(64, filtersize, padding='same'))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization

        # Sixth convolutional block
        model.add(Conv3D(64, filtersize, padding='same'))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization

        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling
        model.add(Dropout(dropout))  # Dropout for regularization

        # Seventh convolutional block
        model.add(Conv3D(128, filtersize, padding='same'))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization

        # Eighth convolutional block
        model.add(Conv3D(128, filtersize, padding='same'))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization

        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling
        model.add(Dropout(dropout))  # Dropout for regularization

        model.add(Flatten())  # Flatten the output from the convolutional layers

        # First dense (fully connected) layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())  # Batch normalization
        model.add(Dropout(dropout))  # Dropout for regularization

        # Second dense (fully connected) layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())  # Batch normalization
        model.add(Dropout(dropout))  # Dropout for regularization

        # Output layer with softmax activation function
        model.add(Dense(self.num_classes, activation='softmax'))

        # Compile the model with Adam optimizer and default learning rate
        optimiser = optimizers.Adam()
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model  # Return the compiled model

In [ ]:
conv_3d5=ModelConv3D5()
conv_3d5.initialize_path(project_folder)
conv_3d5.initialize_image_properties(image_height=120,image_width=120)
conv_3d5.initialize_hyperparams(frames_to_sample=16,batch_size=20,num_epochs=15)
conv_3d5_model=conv_3d5.define_model(filtersize=(3,3,3),dense_neurons=256,dropout=0.25)
conv_3d5_model.summary()

In [ ]:
print("Total Params:", conv_3d5_model.count_params())
history_model5=conv_3d5.train_model(conv_3d5_model,augment_data=True)

In [ ]:
plot(history_model5)


In [ ]:
# Reducing the number of parameters
# Define a new class ModelConv3D6 inheriting from ModelBuilder
class ModelConv3D6(ModelBuilder):
    
    # Define a method to define the model architecture
    def define_model(self, dense_neurons=64, dropout=0.25):

        model = Sequential()  # Initialize a sequential model

        # First convolutional block
        model.add(Conv3D(16, (3, 3, 3), padding='same', input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling

        # Second convolutional block
        model.add(Conv3D(32, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling

        # Third convolutional block
        model.add(Conv3D(64, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling

        # Fourth convolutional block
        model.add(Conv3D(128, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))  # ReLU activation function
        model.add(BatchNormalization())  # Batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Max pooling
        
        model.add(Flatten())  # Flatten the output from the convolutional layers

        # First dense (fully connected) layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())  # Batch normalization
        model.add(Dropout(dropout))  # Dropout for regularization

        # Second dense (fully connected) layer
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())  # Batch normalization
        model.add(Dropout(dropout))  # Dropout for regularization

        # Output layer with softmax activation function
        model.add(Dense(self.num_classes, activation='softmax'))

        # Compile the model with Adam optimizer and a custom learning rate
        optimiser = optimizers.Adam(lr=0.0002)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model  # Return the compiled model

In [ ]:
conv_3d6=ModelConv3D6()
conv_3d6.initialize_path(project_folder)
conv_3d6.initialize_image_properties(image_height=100,image_width=100)
conv_3d6.initialize_hyperparams(frames_to_sample=16,batch_size=20,num_epochs=12)
conv_3d6_model=conv_3d6.define_model(dense_neurons=128,dropout=0.25)
conv_3d6_model.summary()

In [ ]:
print("Total Params:", conv_3d6_model.count_params())
history_model6=conv_3d6.train_model(conv_3d6_model,augment_data=True)

In [ ]:
plot(history_model6)


In [ ]:
# Model 7 - Reducing the number of parameters again

# Define the ModelConv3D7 class that inherits from ModelBuilder
class ModelConv3D7(ModelBuilder):

    # Define the model structure with parameters for dense layer neurons and dropout rate
    def define_model(self, dense_neurons=64, dropout=0.25):

        # Initialize a sequential model
        model = Sequential()
        
        # Add the first 3D convolutional layer with 16 filters, kernel size of 3x3x3, 'same' padding
        # and specify the input shape
        model.add(Conv3D(16, (3, 3, 3), padding='same',
                         input_shape=(self.frames_to_sample, self.image_height, self.image_width, self.channels)))
        model.add(Activation('relu'))  # Add ReLU activation
        model.add(BatchNormalization())  # Add batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Add 3D max pooling layer with pool size of 2x2x2

        # Add the second 3D convolutional layer with 32 filters and kernel size of 3x3x3
        model.add(Conv3D(32, (3, 3, 3), padding='same'))
        model.add(Activation('relu'))  # Add ReLU activation
        model.add(BatchNormalization())  # Add batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Add 3D max pooling layer

        # Add the third 3D convolutional layer with 64 filters and kernel size of 2x2x2
        model.add(Conv3D(64, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))  # Add ReLU activation
        model.add(BatchNormalization())  # Add batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Add 3D max pooling layer

        # Add the fourth 3D convolutional layer with 128 filters and kernel size of 2x2x2
        model.add(Conv3D(128, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))  # Add ReLU activation
        model.add(BatchNormalization())  # Add batch normalization
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))  # Add 3D max pooling layer

        # Flatten the output from the convolutional layers to feed into fully connected layers
        model.add(Flatten())
        
        # Add the first dense layer with the specified number of neurons and ReLU activation
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())  # Add batch normalization
        model.add(Dropout(dropout))  # Add dropout for regularization

        # Add the second dense layer with the same number of neurons and ReLU activation
        model.add(Dense(dense_neurons, activation='relu'))
        model.add(BatchNormalization())  # Add batch normalization
        model.add(Dropout(dropout))  # Add dropout for regularization

        # Add the output layer with the number of classes and softmax activation for classification
        model.add(Dense(self.num_classes, activation='softmax'))

        # Define the optimizer and compile the model with categorical crossentropy loss and accuracy metric
        optimiser = optimizers.Adam(lr=0.0002)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        # Return the compiled model
        return model

In [ ]:
conv_3d7=ModelConv3D7()
conv_3d7.initialize_path(project_folder)
conv_3d7.initialize_image_properties(image_height=120,image_width=120)
conv_3d7.initialize_hyperparams(frames_to_sample=16,batch_size=20,num_epochs=25)
conv_3d7_model=conv_3d7.define_model(dense_neurons=64,dropout=0.25)
conv_3d7_model.summary()

In [ ]:
print("Total Params:", conv_3d7_model.count_params())
history_model7=conv_3d7.train_model(conv_3d7_model,augment_data=True)

In [ ]:
plot(history_model7)
